In [1]:
import os
import autogen
from autogen import ConversableAgent,AssistantAgent, UserProxyAgent

## AGENTS

it is an entity that can send and receive messages to and from other agents in environment which can be powered by models such as GPT-4, code excutors and humans

![alt text](conversable-agent-58656a6f9b08b1d7d942a358e22a28bb.jpg)

The agent has:

1. list of LLM's
2. code executor
3. function and tool executor
4. component for keeping human in loop

In [2]:
agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list":[{"model":"gpt-4","api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},   
    code_execution_config=False, #To turn off code execution
    function_map=None, #No functions registered
    human_input_mode="NEVER" #Never ask for human
)

In [3]:
reply = agent.generate_reply(messages=[{"content": "Tell me a joke","role":"user"}])

In [4]:
print(reply)

Sure, here is a light-hearted joke for you:

Why don't scientists trust atoms?

Because they make up everything!


## Roles and Conversation

we can assign roles to agents and make them participate in the conversation or chat with each other.


In [5]:
cathy=ConversableAgent(
    "cathy",
    system_message="You name is cathy and you are a part of a duo of autogen based model", # defines the initial context of the agent
    llm_config={"config_list":[{"model":"gpt-4","temperature":0.9,"api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]}, # temperature is used for randomness of the model 
    human_input_mode="NEVER",
)
adam=ConversableAgent(
    "adam",
    system_message="You name is cathy and you are a part of a duo of autogen based model", # defines the initial context of the agent
    llm_config={"config_list":[{"model":"gpt-4","temperature":0.6,"api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]}, # temperature is used for randomness of the model 
    human_input_mode="NEVER",
)

# CHAT TERMINATION

termination of chat is important because in any complex, autonomous workflows it is crrucial to stop the workflow 

## Specify parameter in **intiate_chat** keyword :

we define parameters when the conversation starts which determine when it should end

`max_turn` : used to limit the number of turns 



In [6]:
result = adam.initiate_chat(cathy, message="Cathy, what is autogen",max_turns=3) #max_turn limits the conversation exchange to 3

adam (to cathy):

Cathy, what is autogen

--------------------------------------------------------------------------------
cathy (to adam):

Autogen is a tool used to simplify the creation and maintenance of programs that contain large amounts of repetitious text. It is especially valuable in programs that have several blocks of text that must be kept synchronized. 

In machine learning context, this term might be used to describe models or algorithms that can generate automatic responses, processes, or decisions based on the provided data. It's commonly associated with AI-based tools, including chatbots and digital assistants.

--------------------------------------------------------------------------------
adam (to cathy):

That's correct, Cathy. The autogen-based models are used in a variety of fields, including natural language processing, image recognition, and data analysis. They can learn from their mistakes, improve over time, and provide accurate results. Our duo is a good exa

## Agent-triggered termination:

we can also terminate a conversation using parameters to agent 

using `max_consecutive_auto_reply` : condition is triggered when the number of automatic responses to same sender exceeds a threshold

In [7]:
lim=ConversableAgent(
    "lim",
    system_message="your name is lim and you are a comedian",
    llm_config={"config_list":[{"model":"gpt-4","temperature":1,"api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,#limits the number of consecutive auto-replies
)

In [8]:
result=lim.initiate_chat(cathy,message="cathy tell me a joke")

lim (to cathy):

cathy tell me a joke

--------------------------------------------------------------------------------
cathy (to lim):

Sure, here you go:

Why don't scientists trust atoms?

Because they make up everything!

--------------------------------------------------------------------------------
lim (to cathy):

Oh that's a classic one, Cathy! Interested in science jokes, huh? Let me share one too:

Why can't you trust an atom?

Because they literally make up everything!

--------------------------------------------------------------------------------
cathy (to lim):

That's a great one too, Lim! We both seem to enjoy a good atom joke.

--------------------------------------------------------------------------------


using `is_terminated_msg` : a termination message is used to end the conversation

In [ ]:
raz=ConversableAgent(
    "raz",
    system_message="You name is raz and you are a comedian",
    llm_config={"config_list":[{"model":"gpt-4","temperature":0.9,"api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]}, # temperature is used for randomness of the model 
    human_input_mode="NEVER",
    is_termination_msg=lambda msg:"good bye" in msg["content"].lower(),
)

In [10]:
result=lim.initiate_chat(cathy,message="cathy tell me a joke and then say the words GOOD BYE.")

lim (to cathy):

cathy tell me a joke and then say the words GOOD BYE.

--------------------------------------------------------------------------------
cathy (to lim):

Sure, here's a joke for you. Why don't scientists trust atoms? Because they make up everything! 

And now, GOOD BYE.

--------------------------------------------------------------------------------
lim (to cathy):

Thank you, Cathy! Remember, laughter is the best medicine! Have a great one. Goodbye!

--------------------------------------------------------------------------------
cathy (to lim):

You're absolutely right! Laughter is indeed the best medicine. I'm happy I could bring a smile to your face. Have a wonderful time! Goodbye!

--------------------------------------------------------------------------------


# ALLOWING HUMAN FEEDBACK IN AGENTS

the human-in-the-loop components sits in front of auto reply components.

![alt text](human-in-the-loop-56dc002683ab4481042ffd3d346d1759.png)

the human-in-the-loop components can be customized through the `human_input_mode` parameter

### Human input modes
1. Never : no input from human
2. Terminate : only requirs input from human if he decides to terminate or chooses to intercept and reply
3. Always : human reply is always needed

using input mode = `Never`

In [ ]:
import random

# Generate a random number between 1 and 100 (or any range you prefer)
num = random.randint(1, 100)

# Define the ConversableAgent
agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. You have the "
                   f"number {num} in your mind, and I will try to guess it. "
                   "If I guess too high, say 'too high', if I guess too low, say 'too low'.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    is_termination_msg=lambda msg: "53" in msg["content"],  # Terminate if the number is guessed
    human_input_mode="NEVER",  # Never ask for human input
)

In [15]:
agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    human_input_mode="NEVER",
)

In [ ]:
result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

using input mode = `Always`

In [ ]:
human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)

# Start a chat with the agent with number with an initial guess.
result = human_proxy.initiate_chat(
    agent_with_number,  # this is the same agent with the number as before
    message="35",
)

# Code Executors

it is a component that takes input message, performs execution and output messages with results

autogen provides 2 ways to execute code: 
1. command line code executor 
2. unix shell
3. jupyter kernel

In [19]:
import tempfile

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

In [20]:
message_with_code_block = """This is a message with code block.
The code block is below:
```python
import numpy as np
import matplotlib.pyplot as plt
x = np.random.randint(0, 100, 100)
y = np.random.randint(0, 100, 100)
plt.scatter(x, y)
plt.savefig('scatter.png')
print('Scatter plot saved to scatter.png')
```
This is the end of the message.
"""

# Generate a reply for the given code.
reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Scatter plot saved to scatter.png



In [21]:
print(os.listdir(temp_dir.name))

['scatter.png', 'tmp_code_e24bf32d4a21990fb9e4b5eb889ebe5a.py']


In [22]:
temp_dir.cleanup()

### Docker Execution

code : `autogen.coding.DockerCommandLineCodeExecutor`



### Local execution

code : `autogen.coding.LocalCommandLineCodeExecutor`

![alt text](code-executor-no-docker-0ec717a62637ff02e852efa284dd50d5.png)

In [28]:
from autogen.coding import DockerCommandLineCodeExecutor

# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a Docker command line code executor.
executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # Execute code using the given docker image name.
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration that uses docker.
code_executor_agent_using_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the docker command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

# When the code executor is no longer used, stop it to release the resources.
# executor.stop()

### Use code execution in conversation

coding can be a conversation between a code writer agent and a code executor agent, mirroring the interaction between a programmer and a code interpreter.

![alt text](code-execution-in-conversation-f02c7a3ea7e45e3f4aa71d8def851677.png)

In [29]:
# The code writer agent's system message is to instruct the LLM on how to use
# the code executor in the code executor agent.
code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""

code_writer_agent = ConversableAgent(
    "code_writer_agent",
    system_message=code_writer_system_message,
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    code_execution_config=False,  # Turn off code execution for this agent.
)

In [31]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message="Write Python code to calculate the 14th Fibonacci number.",
)

code_executor_agent (to code_writer_agent):

Write Python code to calculate the 14th Fibonacci number.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer_agent (to code_executor_agent):

Sure, let's use Python to calculate the Fibonacci sequence until the 14th number. Fibonacci sequence starts with 0 and 1 then each number is the sum of the two preceding ones.

```python
def fibonacci(n):
    if n <= 0:
        return None
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(n-2):
            a, b = b, a + b
        return b

print(fibonacci(14))
```
This Python script first creates the base cases for the Fibonacci series (where n=1 or n=2). For n above 2, it enters a loop in which each new Fibonacci number is calculated as the sum of the two preceding ones. It finally prints the 14th number in sequence.

------------------------------------------

to get the the stock price gains year-to-date for Tesla and Meta

In [32]:
import datetime

today = datetime.datetime.now().strftime("%Y-%m-%d")
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=f"Today is {today}. Write Python code to plot TSLA's and META's "
    "stock price gains YTD, and save the plot to a file named 'stock_gains.png'.",
)

code_executor_agent (to code_writer_agent):

Today is 2024-12-10. Write Python code to plot TSLA's and META's stock price gains YTD, and save the plot to a file named 'stock_gains.png'.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer_agent (to code_executor_agent):

Sure, we can use data from a financial API like Yahoo Finance to fetch the historical price data of the stocks. Python's `yfinance`, `matplotlib`, `datetime` and `pandas` libraries can be used to achieve that. If these libraries are not installed on your environment please install them using pip - `pip install yfinance matplotlib pandas`.

Here's a Python script to plot the YTD gains for TSLA and META and save it as an image file:

```python
# filename: stock_price_plot.py

import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import pandas as pd

# Define the ticker symbols
tickers = ['TSLA', 'META']

# De

In [34]:
from IPython.display import Image

Image(os.path.join(temp_dir, "stock_gains.png"))

TypeError: expected str, bytes or os.PathLike object, not TemporaryDirectory

In [35]:
temp_dir.cleanup()

In [36]:
executor.stop()  # Stop the docker command line code executor.

# Tool use

Tools are pre-defined functions that agents can use. 

Instead of writing arbitrary code, agents can call tools to perform actions, such as searching the web, performing calculations, reading files, or calling remote APIs.

### Creating tools

In [37]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

### Registering tools

In [ ]:
# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="calculator")(calculator)

<function __main__.calculator(a: int, b: int, operator: Annotated[Literal['+', '-', '*', '/'], 'operator']) -> int>

In [39]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

In [40]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (44232 + 13312 / (232 - 32)) * 5?")

User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_mlHgcKOf5R389CkCaGn0ZTC3): calculator *****
Arguments: 
{
  "a": 232,
  "b": 32,
  "operator": "-"
}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

***** Response from calling tool (call_mlHgcKOf5R389CkCaGn0ZTC3) *****
200
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_RAic0OyfnYP24CFw88hnZwdQ): calculator *****
Arguments: 
{
  "a": 13312,
  "b": 200,
  "operator": "/"
}
******************************

### Tool Schema

In [41]:
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A simple calculator',
   'name': 'calculator',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer', 'description': 'a'},
     'b': {'type': 'integer', 'description': 'b'},
     'operator': {'enum': ['+', '-', '*', '/'],
      'type': 'string',
      'description': 'operator'}},
    'required': ['a', 'b', 'operator']}}}]

In [42]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    a: Annotated[int, Field(description="The first number.")]
    b: Annotated[int, Field(description="The second number.")]
    operator: Annotated[Operator, Field(description="The operator.")]


def calculator(input: Annotated[CalculatorInput, "Input to the calculator."]) -> int:
    if input.operator == "+":
        return input.a + input.b
    elif input.operator == "-":
        return input.a - input.b
    elif input.operator == "*":
        return input.a * input.b
    elif input.operator == "/":
        return int(input.a / input.b)
    else:
        raise ValueError("Invalid operator")

In [43]:
assistant.register_for_llm(name="calculator", description="A calculator tool that accepts nested expression as input")(
    calculator
)
user_proxy.register_for_execution(name="calculator")(calculator)

<function __main__.calculator(input: typing.Annotated[__main__.CalculatorInput, 'Input to the calculator.']) -> int>

In [44]:
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A calculator tool that accepts nested expression as input',
   'name': 'calculator',
   'parameters': {'type': 'object',
    'properties': {'input': {'properties': {'a': {'description': 'The first number.',
        'title': 'A',
        'type': 'integer'},
       'b': {'description': 'The second number.',
        'title': 'B',
        'type': 'integer'},
       'operator': {'description': 'The operator.',
        'enum': ['+', '-', '*', '/'],
        'title': 'Operator',
        'type': 'string'}},
      'required': ['a', 'b', 'operator'],
      'title': 'CalculatorInput',
      'type': 'object',
      'description': 'Input to the calculator.'}},
    'required': ['input']}}}]

In [45]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (1423 - 123) / 3 + (32 + 23) * 5?")

User (to Assistant):

What is (1423 - 123) / 3 + (32 + 23) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_bwHXKTFjtw0HydrDwTtZd6oD): calculator *****
Arguments: 
{
  "input": {
    "a": 1423,
    "b": 123,
    "operator": "-"
  }
}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

***** Response from calling tool (call_bwHXKTFjtw0HydrDwTtZd6oD) *****
1300
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_VwaNDDfafz0ln1p48DpfWhsw): calculator *****
Arguments: 
{
  "input": {
    "a": 1300,
    "b": 3,
    "opera

# Conversation patterns 

1. Two-agent chat:

     the simplest form of conversation pattern where two agents chat with each other
2. Sequential chat:

     a sequence of chats between two agents, chained together by a carryover mechanism, which brings the summary of the previous chat to the context of the next chat.
3. Group Chat: 

     a single chat involving more than two agents. An important question in group chat is: What agent should be next to speak? 
     To support different scenarios, we provide different ways to organize agents in a group chat:

### Two-Agent Chat and Chat Result

Two-agent chat is the simplest form of conversation pattern. We start a two-agent chat using the `initiate_chat` method of every `ConversableAgent` agent. 

![alt text](two-agent-chat-512f70f6b1393b05dc9019c322b15213.png)

A two-agent chats takes two inputs: a message, which is a string provided by the caller; a context, which specifies various parameters of the chat.

The sender agent uses its chat initializer method  to generate an initial message from the inputs, and sends it to the recipient agent to start the chat. 

The sender agent is the agent whose initiate_chat method is called, and the recipient agent is the other agent.

In [46]:
import os

from autogen import ConversableAgent

student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
)
teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    system_message="You are a math teacher.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
)

chat_result = student_agent.initiate_chat(
    teacher_agent,
    message="What is triangle inequality?",
    summary_method="reflection_with_llm",
    max_turns=2,
)

Student_Agent (to Teacher_Agent):

What is triangle inequality?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Teacher_Agent (to Student_Agent):

The triangle inequality theorem states that the sum of the lengths of any two sides of a triangle is always greater than the length of the third side. This theorem is also known as the triangle inequality property and is fundamental to the study of distances in both geometry and numeric analysis. It's crucial for understanding the concept and qualities of Euclidean spaces.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Student_Agent (to Teacher_Agent):

Thank you for your explanation. I understand that this theorem is used to determine if a triangle can be formed from three given sides. Can you please provide a practical example?

--------------------------------------------------------------------------------

>>>

In [47]:
print(chat_result.summary)

The triangle inequality theorem states that the sum of the lengths of any two sides of a triangle is always greater than the length of the third side. It's a fundamental concept in geometry and numeric analysis and can be used practically to determine if a triangle can be formed from three given lengths. For example, three sticks of lengths 3 cm, 4 cm, and 7 cm cannot form a triangle, because the sum of the lengths of two of the sides (3 + 4) is not greater than the third side (7). But, if the lengths are 3 cm, 4 cm, and 5 cm, a triangle can be formed, as for all pairs, the sum of two sides is greater than the remaining side.


In [48]:
print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

Summarize the takeaway from the conversation. Do not add any introductory phrases.


In [49]:
# Get the chat history.
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': 'What is triangle inequality?',
  'name': 'Student_Agent',
  'role': 'assistant'},
 {'content': 'The triangle inequality theorem states that the sum of the '
             'lengths of any two sides of a triangle is always greater than '
             'the length of the third side. This theorem is also known as the '
             'triangle inequality property and is fundamental to the study of '
             "distances in both geometry and numeric analysis. It's crucial "
             'for understanding the concept and qualities of Euclidean spaces.',
  'name': 'Teacher_Agent',
  'role': 'user'},
 {'content': 'Thank you for your explanation. I understand that this theorem '
             'is used to determine if a triangle can be formed from three '
             'given sides. Can you please provide a practical example?',
  'name': 'Student_Agent',
  'role': 'assistant'},
 {'content': "Absolutely! Let's consider this scenario:\n"
             '\n'
             'Suppose you have

In [50]:
# Get the cost of the chat.
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-4-0613': {'completion_tokens': 511,
                                                     'cost': 0.05151,
                                                     'prompt_tokens': 695,
                                                     'total_tokens': 1206},
                                      'total_cost': 0.05151},
 'usage_including_cached_inference': {'gpt-4-0613': {'completion_tokens': 511,
                                                     'cost': 0.05151,
                                                     'prompt_tokens': 695,
                                                     'total_tokens': 1206},
                                      'total_cost': 0.05151}}


### Sequential Chats

The name of this pattern is self-explanatory – it is a sequence of chats between two agents, chained together by a mechanism called carryover, which brings the summary of the previous chat to the context of the next chat.

This pattern is useful for complex task that can be broken down into interdependent sub-tasks. T

![alt text](sequential-two-agent-chat-804707a28b6a1aea61d4a700dc210595.png)

In this pattern, the a pair of agents first start a two-agent chat, then the summary of the conversation becomes a carryover for the next two-agent chat. The next chat passes the carryover to the carryover parameter of the context to generate its initial message.

Carryover accumulates as the conversation moves forward, so each subsequent chat starts with all the carryovers from previous chats.

The figure above shows distinct recipient agents for all the chats, however, the recipient agents in the sequence are allowed to repeat.

In [51]:
# The Number Agent always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    human_input_mode="NEVER",
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    human_input_mode="NEVER",
)

In [52]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.
chat_results = number_agent.initiate_chats(
    [
        {
            "recipient": adder_agent,
            "message": "14",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": multiplier_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": subtracter_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": divider_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Adder_Agent):

14

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

15

--------------------------------------------------------------------------------
Number_Agent (to Adder_Agent):

15

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

16

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Multiplier_Agent):

These are my numbers
Context: 
16

--------------------------------------------------------------------------------
Mul

In [53]:
print("First Chat Summary: ", chat_results[0].summary)
print("Second Chat Summary: ", chat_results[1].summary)
print("Third Chat Summary: ", chat_results[2].summary)
print("Fourth Chat Summary: ", chat_results[3].summary)

First Chat Summary:  16
Second Chat Summary:  64
Third Chat Summary:  13
61
Fourth Chat Summary:  10


### Group chat

AutoGen provides a more general conversation pattern called group chat, which involves more than two agents. 

The core idea of group chat is that all agents contribute to a single conversation thread and share the same context. 

This is useful for tasks that require collaboration among multiple agents.

![alt text](group-chat-2c88a3b4769d60898d0d4d2313d2d168.png)

A group chat is orchestrated by a special agent type `GroupChatManager`

In the first step of the group chat, the Group Chat Manager selects an agent to speak. 

Then, the selected agent speaks and the message is sent back to the Group Chat Manager, who broadcasts the message to all other agents in the group. 

This process repeats until the conversation stops.

The Group Chat Manager can use several strategies to select the next agent. Currently, the following strategies are supported:

`round_robin`: The Group Chat Manager selects agents in a round-robin fashion based on the order of the agents provided.

`random`: The Group Chat Manager selects agents randomly.

`manual`: The Group Chat Manager selects agents by asking for human input.

`auto`: The default strategy, which selects agents using the Group Chat Manager’s LLM.

In [54]:
# The `description` attribute is a string that describes the agent.
# It can also be set in `ConversableAgent` constructor.
adder_agent.description = "Add 1 to each input number."
multiplier_agent.description = "Multiply each input number by 2."
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."
number_agent.description = "Return the numbers given."

In [55]:
from autogen import GroupChat

group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
)

In [56]:
from autogen import GroupChatManager

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
)

In [57]:
result=number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, i want to turn it into 13",
    summary_method="reflection_with_llm",
    )

Number_Agent (to chat_manager):

My number is 3, i want to turn it into 13

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

6

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

7

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

14

--------------------------------------------------------------------------------

Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

13

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

13

--------------------------------------------------------------------------------


In [58]:
print(result.summary)

The agents successfully helped to turn the initial number 3 into 13 through a series of mathematical operations. The multiplier agent multiplied the original number by 6, resulting in 18. The adder agent then added 7 to this, giving 25. The multiplier agent multiplied this by 14 to give 350. Lastly, the subtracter agent subtracted 337, resulting in the final number 13. The number agent confirmed the final result is 13.


#### Send Introductions

he description of the agents to help the Group Chat Manager select the next agent. 

This only helps the Group Chat Manager, however, does not help the participating agents to know about each other. 

Sometimes it is useful have each agent introduce themselves to other agents in the group chat. This can be done by setting the `send_introductions=True`.

In [59]:
group_chat_with_introductions = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
    send_introductions=True,
)

#### Group Chat in a Sequential Chat


Group chat can also be used as a part of a sequential chat. In this case, the Group Chat Manager is treated as a regular agent in the sequence of two-agent chats.

In [60]:
# Let's use the group chat with introduction messages created above.
group_chat_manager_with_intros = GroupChatManager(
    groupchat=group_chat_with_introductions,
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
)

# Start a sequence of two-agent chats between the number agent and
# the group chat manager.
chat_result = number_agent.initiate_chats(
    [
        {
            "recipient": group_chat_manager_with_intros,
            "message": "My number is 3, I want to turn it into 13.",
        },
        {
            "recipient": group_chat_manager_with_intros,
            "message": "Turn this number to 32.",
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------


C:\Users\sides\AppData\Roaming\Python\Python311\site-packages\autogen\agentchat\chat.py:53: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(



Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

6

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

8

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

5

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

10

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to chat_manager):

Turn this number to 32.
Cont

#### Constrained Speaker Selection
Group chat is a powerful conversation pattern, but it can be hard to control if the number of participating agents is large. AutoGen provides a way to constrain the selection of the next speaker by using the `allowed_or_disallowed_speaker_transitions` argument of the GroupChat class.

The `allowed_or_disallowed_speaker_transitions` argument is a dictionary that maps a given agent to a list of agents that can (or cannot) be selected to speak next. The `speaker_transitions_type` argument specifies whether the transitions are allowed or disallowed.

In [61]:
allowed_transitions = {
    number_agent: [adder_agent, number_agent],
    adder_agent: [multiplier_agent, number_agent],
    subtracter_agent: [divider_agent, number_agent],
    multiplier_agent: [subtracter_agent, number_agent],
    divider_agent: [adder_agent, number_agent],
}

In [62]:
constrained_graph_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
    messages=[],
    max_round=12,
    send_introductions=True,
)

constrained_group_chat_manager = GroupChatManager(
    groupchat=constrained_graph_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
)

chat_result = number_agent.initiate_chat(
    constrained_group_chat_manager,
    message="My number is 3, I want to turn it into 10. Once I get to 10, keep it there.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 10. Once I get to 10, keep it there.

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

8

--------------------------------------------------------------------------------

Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

7

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

10

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

11

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

10


### Nested Chats

Nested chats is powered by the nested chats handler, which is a pluggable component of ConversableAgent. 

![alt text](nested-chats-857c3a6a2115d53d588e4fea09d5daf7.png)

When a message comes in and passes the human-in-the-loop component, the nested chats handler checks if the message should trigger a nested chat based on conditions specified by the user. If the conditions are met, the nested chats handler starts a sequence of nested chats specified using the sequential chats pattern. In each of the nested chats, the sender agent is always the same agent that triggered the nested chats. In the end, the nested chat handler uses the results of the nested chats to produce a response to the original message. By default, the nested chat handler uses the summary of the last chat as the response.

In [63]:
import tempfile

temp_dir = tempfile.gettempdir()

arithmetic_agent = ConversableAgent(
    name="Arithmetic_Agent",
    llm_config=False,
    human_input_mode="ALWAYS",
    # This agent will always require human input to make sure the code is
    # safe to execute.
    code_execution_config={"use_docker": False, "work_dir": temp_dir},
)

code_writer_agent = ConversableAgent(
    name="Code_Writer_Agent",
    system_message="You are a code writer. You write Python script in Markdown code blocks.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    human_input_mode="NEVER",
)

poetry_agent = ConversableAgent(
    name="Poetry_Agent",
    system_message="You are an AI poet.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key":"API_KEY_PLACEHOLDERproj-8jhaENcxBVVg8iAhSEQBT3BlbkFJ7S6mDafUVTonpaZWq7X1"}]},
    human_input_mode="NEVER",
)

In [64]:
nested_chats = [
    {
        "recipient": group_chat_manager_with_intros,
        "summary_method": "reflection_with_llm",
        "summary_prompt": "Summarize the sequence of operations used to turn " "the source number into target number.",
    },
    {
        "recipient": code_writer_agent,
        "message": "Write a Python script to verify the arithmetic operations is correct.",
        "summary_method": "reflection_with_llm",
    },
    {
        "recipient": poetry_agent,
        "message": "Write a poem about it.",
        "max_turns": 1,
        "summary_method": "last_msg",
    },
]

In [65]:
arithmetic_agent.register_nested_chats(
    nested_chats,
    # The trigger function is used to determine if the agent should start the nested chat
    # given the sender agent.
    # In this case, the arithmetic agent will not start the nested chats if the sender is
    # from the nested chats' recipient to avoid recursive calls.
    trigger=lambda sender: sender not in [group_chat_manager_with_intros, code_writer_agent, poetry_agent],
)

In [66]:
# Instead of using `initiate_chat` method to start another conversation,
# we can use the `generate_reply` method to get single reply to a message directly.
reply = arithmetic_agent.generate_reply(
    messages=[{"role": "user", "content": "I have a number 3 and I want to turn it into 7."}]
)


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

********************************************************************************
Starting a new chat....

********************************************************************************
Arithmetic_Agent (to chat_manager):

I have a number 3 and I want to turn it into 7.

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (to chat_manager):

I believe you might have some confusion here. As the "Adder_Agent", I can only add 1 to the number you provide. So if you provide the number 3, I can turn it into 4. Is that the action you'd like me to take? I can't turn 3 into 7 by adding 1.

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

6

--------------------------------------------------------------------------------

Next speaker: Adder_Agent

Adder_Agent (

Chat with LLMs: In Human in the Loop we covered the basic human-in-the-loop usage. You can try to hook up different LLMs using local model servers like Ollama and LM Studio, and chat with them using the human-in-the-loop component of your human proxy agent.

Prompt Engineering: In Code Executors we covered the simple two agent scenario using GPT-4 and Python code executor. To make this scenario work for different LLMs and programming languages, you probably need to tune the system message of the code writer agent. Same with other scenarios that we have covered in this tutorial, you can also try to tune system messages for different LLMs.

Complex Tasks: In ConversationPatterns we covered the basic conversation patterns. You can try to find other tasks that can be decomposed into these patterns, and leverage the code executors and tools to make the agents more powerful.